## Pytorch Neural Network

In [1]:
"""
x-Inputs
wx-weights
bx-bias
a-activation
y-Output
"""

'\nx-Inputs\nwx-weights\nbx-bias\na-activation\ny-Output\n'

## Components of pytorch

- Base class for defining customer models: torch.nn.Module
- Fully connected (dense) layers: torch.nn.Linear
- Activation function: torch.nn.ReLu
- Optimizer: torch.optim
- loss function: torch.nn.CrossEntropyLoss
- Loads data in batches : torch.utils.data.Dataloader

## Different ways to create neural network
1. Functional : Flexible, harder to interpret
2. Sequential : nn.sequential ,only for simple architecture without resnets

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
## Functional API
class Simplenn(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(Simplenn,self).__init__()

        self.fc1=nn.Linear(input_size,hidden_size)

        self.relu=nn.ReLU()

        self.fc2=nn.Linear(hidden_size,output_size)
    def forward(self,x):
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        x=self.relu(x)
        return x


In [ ]:
#sequential api
class SimpleNNSequential(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(SimpleNNSequential,self).__init__()
        self.network=nn.Sequential(nn.Linear(input_size,hidden_size),
                      nn.ReLU(),
                      nn.Linear(hidden_size,output_size),
                      )
    def forward(self,x):
        x=self.network(x)
        return x

# Training Neural network

In [15]:
model_func=Simplenn(input_size=4,hidden_size=8,output_size=3)
print(model_func)

Simplenn(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=8, out_features=3, bias=True)
)


In [ ]:
x=torch.randn(10,4) #ten samples 4 features
y=torch.randint(0,3,(10,))
criteria=nn.CrossEntropyLoss()
optimizer=optim.Adam(model_func.parameters(),lr=0.01)

In [ ]:
print(x) # four rows, ten data points

tensor([[-0.6559, -0.3644,  1.1038,  2.4157],
        [-0.0985, -0.3709,  0.1421,  0.9090],
        [-0.4295, -3.2971,  0.9990, -0.8835],
        [-0.0816, -0.6280,  0.0296,  0.7199],
        [ 0.0543,  1.1725, -1.6407,  2.0513],
        [-1.1735,  1.6088,  0.5560,  0.7541],
        [ 1.1261, -0.2264,  0.2936, -0.2462],
        [-1.1367,  0.9988, -0.4802, -0.8654],
        [-0.8584, -0.4736, -0.0160, -0.3988],
        [ 0.9344,  2.5322,  1.9835,  0.9854]])


In [18]:
print(y)

tensor([2, 1, 1, 0, 1, 2, 0, 2, 0, 2])


In [21]:
# training loop
epoch=50

for e in range(epoch):
    optimizer.zero_grad()
    outputs=model_func(x)
    loss=criteria(outputs,y)
    loss.backward()
    optimizer.step()

    if (e+1)%10 == 0:
        print(f"epoch: {e+1}/50,loss:{loss.item():.4f}")
     


epoch: 10/50,loss:0.1743
epoch: 20/50,loss:0.1535
epoch: 30/50,loss:0.1343
epoch: 40/50,loss:0.1166
epoch: 50/50,loss:0.1004


## Regression model with pytorch components 

In [1]:
# Data gathering
# Data preprocessing
# feature engineering
# model training
# testing

In [1]:
import kagglehub
path=kagglehub.dataset_download("mirichoi0218/insurance")
print(path)

c:\Users\girim\anaconda3\envs\CV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


C:\Users\girim\.cache\kagglehub\datasets\mirichoi0218\insurance\versions\1


In [2]:
import pandas as pd
import os
os.listdir(r'C:\Users\girim\.cache\kagglehub\datasets\mirichoi0218\insurance\versions\1')

['insurance.csv']

In [3]:
df=pd.read_csv(os.path.join(path,'insurance.csv'))

In [4]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split

In [37]:
#split data set before encoding
train_df,test_df=train_test_split(df,test_size=0.2,random_state=42)


In [38]:
label_encoder={}
for col in ['sex','smoker','region']:
    le=LabelEncoder()
    train_df[col]=le.fit_transform(train_df[col])
    test_df[col]=le.transform(test_df[col])
    label_encoder[col]=le

In [39]:
label_encoder

{'sex': LabelEncoder(), 'smoker': LabelEncoder(), 'region': LabelEncoder()}

In [40]:
# feature and target
y_train=train_df['charges']
x_train=train_df.drop(['charges'],axis=1)

y_test=test_df['charges']
x_test=test_df.drop(['charges'],axis=1)

In [41]:
x_train.shape

(1070, 6)

In [42]:
# Normalize features
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [43]:
# Convert to tensor
x_train_tensor=torch.tensor(x_train,dtype=torch.float32)
x_test_tensor=torch.tensor(x_test,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train.values,dtype=torch.float32).view(-1,1)
y_test_tensor=torch.tensor(y_test.values,dtype=torch.float32).view(-1,1)

In [44]:
y_train_tensor.shape

torch.Size([1070, 1])

In [45]:
# Define Neural Network

class SimpleNNRegression(nn.Module):
    def __init__(self,input_dim):
        super(SimpleNNRegression,self).__init__()
        self.network=nn.Sequential(nn.Linear(input_dim,64),nn.ReLU(),
                                   nn.Linear(64,128),nn.ReLU(),
                                   nn.Linear(128,1))
    def forward(self,x):
        return self.network(x)

In [46]:
model=SimpleNNRegression(input_dim=6) #input tensor has 6 columns

In [47]:
# L0ss and criteria
criteria=nn.MSELoss()
optimizer=optim.Adam(model.parameters())

In [54]:
#Training loop
epochs=30000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    pred=model(x_train_tensor)
    loss=criteria(pred,y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch+1)%100==0:
        print(f"{epoch+1}/30000 loss : {loss.item():.4f}")

100/30000 loss : 4473963.5000
200/30000 loss : 4461280.5000
300/30000 loss : 4448267.5000
400/30000 loss : 4435152.0000
500/30000 loss : 4422665.5000
600/30000 loss : 4410045.5000
700/30000 loss : 4397562.0000
800/30000 loss : 4385468.5000
900/30000 loss : 4372856.5000
1000/30000 loss : 4360830.0000
1100/30000 loss : 4347408.0000
1200/30000 loss : 4334096.5000
1300/30000 loss : 4321013.0000
1400/30000 loss : 4307136.5000
1500/30000 loss : 4294179.0000
1600/30000 loss : 4281327.5000
1700/30000 loss : 4269156.0000
1800/30000 loss : 4257312.0000
1900/30000 loss : 4243673.0000
2000/30000 loss : 4231185.0000
2100/30000 loss : 4219046.0000
2200/30000 loss : 4206857.5000
2300/30000 loss : 4193650.0000
2400/30000 loss : 4180479.0000
2500/30000 loss : 4167258.5000
2600/30000 loss : 4154176.0000
2700/30000 loss : 4142577.2500
2800/30000 loss : 4130528.0000
2900/30000 loss : 4118347.2500
3000/30000 loss : 4105877.2500
3100/30000 loss : 4092057.0000
3200/30000 loss : 4079891.5000
3300/30000 loss :

In [55]:
model.eval()
y_pred=model(x_test_tensor).detach().numpy()

In [56]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

y_test_numpy=y_test_tensor.numpy()
# Calculate Metrics
mse=mean_squared_error(y_test_numpy,y_pred)
mae=mean_absolute_error(y_test_numpy,y_pred)
r2=r2_score(y_test_numpy,y_pred)

print(mse)
print(mae)
print(r2)
#43

46042712.0
4699.53271484375
0.7034261226654053


In [57]:
def predict_charges(age,sex,bmi,children,smoker,region):
    input_data=pd.DataFrame([[age,sex,bmi,children,smoker,region]],columns=['age','sex','bmi','children','smoker','region'])
    for col in ['sex','smoker','region']:
        input_data[col]=label_encoder[col].transform(input_data[col])
    input_data=scaler.transform(input_data)
    input_tensor=torch.tensor(input_data,dtype=torch.float32)
    predicted_charge=model(input_tensor).item()
    return predicted_charge

In [60]:
predicticted=predict_charges(37,'female',27.74,3,'no','northwest')

In [61]:
print(predicticted)

6186.333984375
